### NHL API's boxscore endpoint to a Pandas DataFrame

Example URL for a single game: https://statsapi.web.nhl.com/api/v1/game/2017020001/boxscore.
Where in 2017020001:
* first 4 digits are the year (2017-2018 season)
* next 2 digits are the type of season (02 for regular season)
* and the 4 last digits are the Game ID ranging from 0001 to 1271 (for the seasons starting from 2017 w 31 teams)
* Number of games was 1230 from 2000 until 2016-2017

Runtime for one season is 13-14 minutes

In [8]:
import requests
import pandas as pd
import csv

pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 12)
pd.set_option('display.width', 1000)

In [9]:
#===returns a DF with team related data such as team name abbreviation and team ID ===============#
def teamData():
    response = requests.get('https://statsapi.web.nhl.com/api/v1/teams')
    dict_data = response.json()
    teams = dict_data['teams']          #chooses the 'teams' dictionary

    all_teams = []
    for team in teams:
        team_data = []
        team_id = team['id']
        # team_name = team['name']
        team_abbr = team['abbreviation']
        team_data.extend([team_id, team_abbr])
        all_teams.append(team_data)
        
    headers = ['t_id', 't_abbr']
    teamData_df = pd.DataFrame(all_teams, columns = headers)
    
    return teamData_df

In [10]:
#===returns the full "box score" for all regular season games. Has to be done separately for all seasons. ==============#
def boxscore():
    full_game_list = []
    game_ids = []
    
    # 2013-2017: 1230 games
    # 2017 - 2020 1271 games
    for game_id in range(2013020001, 2013021231):
        url = 'https://statsapi.web.nhl.com/api/v1/game/{}/boxscore'.format(game_id)
        request = requests.get(url)
        dict_data = request.json()
        
        one_game = []
        game_id = game_id
        h_id = dict_data['teams']['home']['team']['id']
        a_id = dict_data['teams']['away']['team']['id']
        h_goals = dict_data['teams']['home']['teamStats']['teamSkaterStats']['goals']
        a_goals = dict_data['teams']['away']['teamStats']['teamSkaterStats']['goals']
        h_pim = dict_data['teams']['home']['teamStats']['teamSkaterStats']['pim']
        a_pim = dict_data['teams']['away']['teamStats']['teamSkaterStats']['pim']
        h_shots = dict_data['teams']['home']['teamStats']['teamSkaterStats']['shots']
        a_shots = dict_data['teams']['away']['teamStats']['teamSkaterStats']['shots']
        h_PP_pct = dict_data['teams']['home']['teamStats']['teamSkaterStats']['powerPlayPercentage']
        a_PP_pct = dict_data['teams']['away']['teamStats']['teamSkaterStats']['powerPlayPercentage']
        h_PPG = dict_data['teams']['home']['teamStats']['teamSkaterStats']['powerPlayGoals']
        a_PPG = dict_data['teams']['away']['teamStats']['teamSkaterStats']['powerPlayGoals']
        h_PP_oppt = dict_data['teams']['home']['teamStats']['teamSkaterStats']['powerPlayOpportunities']
        a_PP_oppt = dict_data['teams']['away']['teamStats']['teamSkaterStats']['powerPlayOpportunities']
        h_FO_win_pct = dict_data['teams']['home']['teamStats']['teamSkaterStats']['faceOffWinPercentage']
        a_FO_win_pct = dict_data['teams']['away']['teamStats']['teamSkaterStats']['faceOffWinPercentage']
        h_blocked = dict_data['teams']['home']['teamStats']['teamSkaterStats']['blocked']
        a_blocked = dict_data['teams']['away']['teamStats']['teamSkaterStats']['blocked']
        h_takeaways = dict_data['teams']['home']['teamStats']['teamSkaterStats']['takeaways']
        a_takeaways = dict_data['teams']['away']['teamStats']['teamSkaterStats']['takeaways']
        h_giveaways = dict_data['teams']['home']['teamStats']['teamSkaterStats']['giveaways']
        a_giveaways = dict_data['teams']['away']['teamStats']['teamSkaterStats']['giveaways']
        h_hits = dict_data['teams']['home']['teamStats']['teamSkaterStats']['hits']
        a_hits = dict_data['teams']['away']['teamStats']['teamSkaterStats']['hits']
        
        
        one_game.extend([game_id, h_id, a_id,h_goals, a_goals, h_pim, a_pim, h_shots, a_shots, h_PP_pct,\
                         a_PP_pct, h_PPG, a_PPG, h_PP_oppt, a_PP_oppt, h_FO_win_pct, a_FO_win_pct,\
                         h_blocked, a_blocked, h_takeaways, a_takeaways, h_giveaways, a_giveaways, h_hits, a_hits])
        full_game_list.append(one_game)
        game_ids.append(game_id) #might not need this list

    bs_df = pd.DataFrame(full_game_list, columns=\
                      ['game_id','h_id', 'a_id','h_goals', 'a_goals', 'h_pim', 'a_pim', 'h_shots', 'a_shots', 'h_PP_pct',\
                       'a_PP_pct', 'h_PPG', 'a_PPG', 'h_PP_oppt', 'a_PP_oppt', 'h_FO_pct', 'a_FO_pct',\
                       'h_blocked', 'a_blocked', 'h_tAways', 'a_tAways', 'h_gAways', 'a_gAways', 'h_hits', 'a_hits'])

    return bs_df

In [11]:
#=========Merges team data with box score to get team names and abbrvs to box score===========#
def merge_dfs(bs_df, team_df):
    merged_df = bs_df.merge(team_df, left_on = 'a_id', right_on = 't_id').\
                merge(team_df, left_on = 'h_id', right_on = 't_id').\
                drop(['t_id_x', 't_id_y'], axis = 1)
    
    merged_df = merged_df.rename(columns={'t_abbr_x': 'a_abbr', 't_abbr_y': 'h_abbr'})

    
    merged_df = merged_df[['game_id', 'h_id', 'h_abbr', 'a_id', 'a_abbr','h_goals', 'a_goals', 'h_pim', 'a_pim', 'h_shots', 'a_shots', 'h_PP_pct',\
                       'a_PP_pct', 'h_PPG', 'a_PPG', 'h_PP_oppt', 'a_PP_oppt', 'h_FO_pct', 'a_FO_pct',\
                       'h_blocked', 'a_blocked', 'h_tAways', 'a_tAways', 'h_gAways', 'a_gAways', 'h_hits', 'a_hits']]
    
    merged_df.set_index('game_id', drop=True, inplace=True) #DONT DROP? (default = True)
    return merged_df


In [14]:
#===================DataFrame to a CSV ====================================#

def df_to_csv(df):
    df.to_csv(r'YOUR-FILE-PATH\2013-2014-aug.csv')

In [15]:
def main():
    team_df = teamData()
    bs_df = boxscore()
    df = merge_dfs(bs_df, team_df)
    df_to_csv(df)
    return df

main()

,h_id,h_abbr,a_id,a_abbr,h_goals,a_goals,...,h_tAways,a_tAways,h_gAways,a_gAways,h_hits,a_hits
game_id,,,,,,,,,,,,,
2013020001,8,MTL,10,TOR,3,4,...,7,3,10,5,21,25
2013020394,8,MTL,10,TOR,4,2,...,3,1,8,8,14,21
2013020909,8,MTL,10,TOR,4,3,...,2,2,8,5,22,17
2013020777,8,MTL,15,WSH,0,5,...,6,2,3,7,29,8
2013020836,8,MTL,52,WPG,1,2,...,5,3,6,8,26,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013020156,10,TOR,5,PIT,4,1,...,8,4,13,14,46,26
2013020423,10,TOR,25,DAL,3,2,...,16,8,19,10,37,41
2013020450,10,TOR,6,BOS,2,5,...,6,8,12,9,36,29
